In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from IPython.display import display, HTML
import json

In [2]:
df = pd.DataFrame()

summary_fn = np.mean

exp_path = Path('experiments/HyperparameterOptimized')
for ds in sorted(exp_path.iterdir(), key=lambda x: x.name.lower()):
    if not ds.is_dir():
        continue
    accus = []
    for run in ds.iterdir():
        try:
            with open(run / 'stats.json') as f:
                stats = json.load(f)
            accu_test_stats = stats['accu_test']
            test_accu = [s for s in accu_test_stats if s['epoch'] == 200][-1]
            accus.append(test_accu['value'])
        except Exception:
            pass
    if accus:
        df.at[ds.name, 'ProtoTSNet'] = summary_fn(accus) * 100

exp_path = Path('experiments/NoPretrainingRegularEnc')
for ds in exp_path.iterdir():
    if not ds.is_dir():
        continue
    accus = []
    for run in ds.iterdir():
        try:
            with open(run / 'stats.json') as f:
                stats = json.load(f)
            accu_test_stats = stats['accu_test']
            test_accu = [s for s in accu_test_stats if s['epoch'] == 200][-1]
            accus.append(test_accu['value'])
        except Exception:
            pass
    if accus:
        df.at[ds.name, 'ProtoTSNet (R)'] = summary_fn(accus) * 100

mr_petsc_results = Path('../protoTS/petsc/python/results')
for ds in mr_petsc_results.iterdir():
    cv_results = []
    for run in ds.iterdir():
        with open(run / 'results.json') as f:
            results = json.load(f)
            cv_results.append(results['accu'])
    df.at[ds.name, 'MR-PETSC'] = summary_fn(cv_results) * 100

XCM_results_path = Path('../protoTS/XCM/results')
for ds in XCM_results_path.iterdir():
    if not ds.is_dir():
        continue
    cv_results = []
    try:
        for fold in range(1, 6):
            with open(ds / 'XCM' / f'XP_{fold}' / 'results.json') as f:
                results = json.load(f)
            cv_results.append(results['accu'])
        df.at[ds.name, 'XCM'] = summary_fn(cv_results) * 100
    except Exception:
        pass

XCM_results_path = Path('../protoTS/XCM/results')
for ds in XCM_results_path.iterdir():
    if not ds.is_dir():
        continue
    cv_results = []
    for fold in range(1, 6):
        try:
            with open(ds / 'MTEX-CNN' / f'XP_{fold}' / 'results.json') as f:
                results = json.load(f)
            cv_results.append(results['accu'])
        except Exception:
            pass
    if cv_results:
        df.at[ds.name, 'MTEX-CNN'] = summary_fn(cv_results) * 100

tapnet_results = Path('../protoTS/tapnet/results')
for ds in tapnet_results.iterdir():
    cv_results = []
    for run in ds.iterdir():
        with open(run / 'results.json') as f:
            results = json.load(f)
            cv_results.append(results['test_acc'])
    df.at[ds.name, 'TapNet'] = summary_fn(cv_results) * 100

bakeoff = pd.read_csv('https://timeseriesclassification.com/results/PublishedResults/Bakeoff2021/ROCKET_TESTFOLDS.csv', header='infer', index_col=0)
bakeoff.index.name = ''
rocket_results = bakeoff.iloc[:, 0]
for ds in rocket_results.index:
    df.at[ds, 'ROCKET'] = rocket_results[ds] * 100

# display(HTML(df.to_html()))

unmodified_df = df.copy()
rounded_df = df.astype(float).round(1)

def display_highlight_max(s):
    explainable = s.copy()
    del explainable['TapNet']
    non_ex_ind_st = -1
    if 'ROCKET' in explainable:
        del explainable['ROCKET']
        non_ex_ind_st = -2
    max_explainable = explainable.max()
    is_max_explainable = s == max_explainable
    is_max_explainable[non_ex_ind_st:] = False
    max_overall = s.max()
    is_max_overall = s == max_overall
    return ['{}{}'.format('font-weight: bold; color: #6495ED;' if me else '', 'text-decoration: underline;' if mo else '') for me, mo in zip(is_max_explainable, is_max_overall)]
    
styled_df = rounded_df.astype(float).round(1).style.apply(display_highlight_max, axis=1)
display(HTML(styled_df.format(lambda f: '{:.1f}%'.format(f) if not np.isnan(f) else 'N/A').to_html()))

def latex_bold_max(s):
    explainable = s.copy()
    del explainable['TapNet']
    non_ex_ind_st = -1
    if 'ROCKET' in explainable:
        del explainable['ROCKET']
        non_ex_ind_st = -2
    max_explainable = explainable.max()
    is_max_explainable = s == max_explainable
    is_max_explainable[non_ex_ind_st:] = False
    max_overall = s.max()
    is_max_overall = s == max_overall
    
    def wrap(v, bold, underline):
        v = str(v)
        if bold:
            v = '\\textbf{' + v + '}'
        if underline:
            v = '\\underline{' + v + '}'
        return v
    return ['N/A' if np.isnan(v) else wrap(v, v_is_ex_max, v_is_max) for v, v_is_max, v_is_ex_max in zip(s, is_max_overall, is_max_explainable)]

styled_df = rounded_df.apply(latex_bold_max, axis=1)

styled_df = pd.DataFrame(styled_df.tolist(), columns=rounded_df.columns)
styled_df.index = rounded_df.index
styled_df = styled_df.sort_index(key=lambda x: x.str.lower())

latex_str = styled_df.to_latex(float_format='%.2f', escape=False)

print(latex_str.replace('\n', '\n        '))

,ProtoTSNet,ProtoTSNet (R),MR-PETSC,XCM,MTEX-CNN,TapNet,ROCKET
ArticularyWordRecognition,97.1%,94.7%,99.3%,72.9%,94.2%,98.3%,99.3%
AtrialFibrillation,33.3%,32.0%,30.7%,30.7%,33.3%,25.3%,6.7%
BasicMotions,97.0%,98.5%,94.5%,95.5%,97.0%,100.0%,100.0%
CharacterTrajectories,97.6%,81.5%,93.3%,95.7%,95.8%,99.7%,N/A
Cricket,93.1%,92.8%,99.4%,71.7%,89.4%,97.8%,100.0%
DuckDuckGeese,65.2%,58.0%,42.0%,31.6%,51.2%,48.8%,52.0%
EigenWorms,52.1%,50.7%,N/A,28.2%,42.0%,47.8%,90.8%
Epilepsy,98.4%,68.0%,98.4%,89.7%,92.8%,97.0%,99.3%
ERing,89.9%,86.4%,88.3%,20.6%,84.5%,82.0%,98.5%
EthanolConcentration,24.5%,27.0%,50.6%,28.8%,28.5%,28.0%,44.5%


\begin{tabular}{llllllll}
        \toprule
         & ProtoTSNet & ProtoTSNet (R) & MR-PETSC & XCM & MTEX-CNN & TapNet & ROCKET \\
        \midrule
        ArticularyWordRecognition & 97.1 & 94.7 & \underline{\textbf{99.3}} & 72.9 & 94.2 & 98.3 & \underline{99.3} \\
        AtrialFibrillation & \underline{\textbf{33.3}} & 32.0 & 30.7 & 30.7 & \underline{\textbf{33.3}} & 25.3 & 6.7 \\
        BasicMotions & 97.0 & \textbf{98.5} & 94.5 & 95.5 & 97.0 & \underline{100.0} & \underline{100.0} \\
        CharacterTrajectories & \textbf{97.6} & 81.5 & 93.3 & 95.7 & 95.8 & \underline{99.7} & N/A \\
        Cricket & 93.1 & 92.8 & \textbf{99.4} & 71.7 & 89.4 & 97.8 & \underline{100.0} \\
        DuckDuckGeese & \underline{\textbf{65.2}} & 58.0 & 42.0 & 31.6 & 51.2 & 48.8 & 52.0 \\
        EigenWorms & \textbf{52.1} & 50.7 & N/A & 28.2 & 42.0 & 47.8 & \underline{90.8} \\
        Epilepsy & \textbf{98.4} & 68.0 & \textbf{98.4} & 89.7 & 92.8 & 97.0 & \underline{99.3} \\
        ERing & \textbf{89.9

In [3]:
ranks_df = unmodified_df.rank(axis=1, method='min', ascending=False)
overall_ranks_mean = ranks_df.mean().round(2)
wins = (ranks_df == 1).sum()

explainable_df = unmodified_df.copy()
del explainable_df['TapNet']
del explainable_df['ROCKET']
ranks_df_ex = explainable_df.rank(axis=1, method='min', ascending=False)
explainable_ranks_mean = ranks_df_ex.mean().round(2)
wins_ex = (ranks_df_ex == 1).sum()

summary_ranks_df = pd.concat([pd.DataFrame(overall_ranks_mean).T, pd.DataFrame(wins).T, pd.DataFrame(explainable_ranks_mean).T, pd.DataFrame(wins_ex).T])
summary_ranks_df.index = ['Avg. Rank', 'Wins/Ties', 'Avg. Rank (Explainable)', 'Wins/Ties (Explainable)']
display(HTML(summary_ranks_df.to_html(float_format='%.2f')))

latex_rank_df = pd.DataFrame(columns=df.columns)
latex_rank_df.loc['Avg. Rank'] = ranks_df.mean()
latex_rank_df.loc['Wins/Ties'] = wins

latex_rank_df_ex = pd.DataFrame(columns=df.columns)
latex_rank_df_ex.loc['Avg. Rank'] = ranks_df_ex.mean()
latex_rank_df_ex.loc['Wins/Ties'] = wins_ex

def get_bolder(do_underline):
    def bold_best(s):
        explainable = s.copy()
        del explainable['TapNet']
        non_ex_ind_st = -1
        if 'ROCKET' in explainable:
            del explainable['ROCKET']
            non_ex_ind_st = -2
        best_ex = explainable.min() if s.name == 'Avg. Rank' else explainable.max()
        is_best_ex = s == best_ex
        is_best_ex[non_ex_ind_st:] = False
        best_overall = s.min() if s.name == 'Avg. Rank' else s.max()
        is_best_overall = s == best_overall
        
        def format_float(f):
            return '{:.2f}'.format(f) if s.name == 'Avg. Rank' else '{:.0f}'.format(f)
        
        def wrap(v, bold, underline):
            v = format_float(v)
            if bold and not do_underline:
                v = '\\textbf{' + v + '}'
            if underline and do_underline:
                v = '\\underline{' + v + '}'
            return v
        return ['' if np.isnan(v) else wrap(v, v_is_ex_max, v_is_max) for v, v_is_max, v_is_ex_max in zip(s, is_best_overall, is_best_ex)]
    return bold_best

styled_df = latex_rank_df.apply(get_bolder(do_underline=True), axis=1)

styled_df = pd.DataFrame(styled_df.tolist(), columns=df.columns)
styled_df.index = latex_rank_df.index
styled_df = styled_df.sort_index(key=lambda x: x.str.lower())

latex_str = styled_df.to_latex(float_format='%.2f', escape=False)
print(latex_str.replace('\n', '\n        '))

do_underline = False
styled_df = latex_rank_df_ex.apply(get_bolder(do_underline=False), axis=1)

styled_df = pd.DataFrame(styled_df.tolist(), columns=df.columns)
styled_df.index = latex_rank_df_ex.index
styled_df = styled_df.sort_index(key=lambda x: x.str.lower())

latex_str = styled_df.to_latex(float_format='%.2f', escape=False)
print(latex_str.replace('\n', '\n        ').replace(' &  \\\\', '\\multicolumn{2}{c|}{} \\\\'))


,ProtoTSNet,ProtoTSNet (R),MR-PETSC,XCM,MTEX-CNN,TapNet,ROCKET
Avg. Rank,3.13,4.03,4.16,5.17,4.83,3.87,1.58
Wins/Ties,1.00,0.00,2.00,0.00,2.00,5.00,21.00
Avg. Rank (Explainable),1.97,2.77,2.84,3.63,3.33,NaN,NaN
Wins/Ties (Explainable),13.00,5.00,8.00,2.00,3.00,NaN,NaN


\begin{tabular}{llllllll}
        \toprule
         & ProtoTSNet & ProtoTSNet (R) & MR-PETSC & XCM & MTEX-CNN & TapNet & ROCKET \\
        \midrule
        Avg. Rank & 3.13 & 4.03 & 4.16 & 5.17 & 4.83 & 3.87 & \underline{1.58} \\
        Wins/Ties & 1 & 0 & 2 & 0 & 2 & 5 & \underline{21} \\
        \bottomrule
        \end{tabular}
        
\begin{tabular}{llllllll}
        \toprule
         & ProtoTSNet & ProtoTSNet (R) & MR-PETSC & XCM & MTEX-CNN & TapNet & ROCKET \\
        \midrule
        Avg. Rank & \textbf{1.97} & 2.77 & 2.84 & 3.63 & 3.33 & \multicolumn{2}{c|}{} \\
        Wins/Ties & \textbf{13} & 5 & 8 & 2 & 3 & \multicolumn{2}{c|}{} \\
        \bottomrule
        \end{tabular}
        


In [4]:
ranks_df.mean().round(2).to_csv('ranks.csv', header=False)
ranks_df_ex.mean().round(2).to_csv('ranks_ex.csv', header=False)

In [5]:
f1_df = pd.DataFrame()

def calculate_macro_f1(confusion_matrix):
    num_classes = confusion_matrix.shape[0]
    
    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)
    
    for i in range(num_classes):
        TP = confusion_matrix[i, i]
        FP = confusion_matrix[:, i].sum() - TP
        FN = confusion_matrix[i, :].sum() - TP
        precision[i] = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall[i] = TP / (TP + FN) if (TP + FN) > 0 else 0
    
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8) # Adding a small epsilon to avoid division by zero
    
    macro_f1 = np.mean(f1_scores)
    return macro_f1


exp_path = Path('experiments/HyperparameterOptimized')
for ds in sorted(exp_path.iterdir(), key=lambda x: x.name.lower()):
    if not ds.is_dir() or ds.is_symlink():
        continue
    f1_to_avg = []
    for run in ds.iterdir():
        try:
            conf_mat = np.loadtxt(run / 'confusion_matrix.txt')
            f1_to_avg.append(calculate_macro_f1(conf_mat))
        except Exception as e:
            pass
        if f1_to_avg:
            f1_df.at[ds.name, 'ProtoTSNet'] = np.mean(f1_to_avg)
        else:
            f1_df.at[ds.name, 'ProtoTSNet'] = np.nan


exp_path = Path('experiments/NoPretrainingRegularEnc')
for ds in sorted(exp_path.iterdir(), key=lambda x: x.name.lower()):
    if not ds.is_dir() or ds.is_symlink():
        continue
    f1_to_avg = []
    for run in ds.iterdir():
        try:
            conf_mat = np.loadtxt(run / 'confusion_matrix.txt')
            f1_to_avg.append(calculate_macro_f1(conf_mat))
        except Exception as e:
            pass
        if f1_to_avg:
            f1_df.at[ds.name, 'ProtoTSNet (R)'] = np.mean(f1_to_avg)
        else:
            f1_df.at[ds.name, 'ProtoTSNet (R)'] = np.nan

mr_petsc_results = Path('../protoTS/petsc/python/results')
for ds in mr_petsc_results.iterdir():
    f1_to_avg = []
    for run in ds.iterdir():
        conf_mat = np.loadtxt(run / 'confusion_matrix.txt')
        f1_to_avg.append(calculate_macro_f1(conf_mat))
    f1_df.at[ds.name, 'MR-PETSC'] = np.mean(f1_to_avg)


XCM_results_path = Path('../protoTS/XCM/results')
for ds in XCM_results_path.iterdir():
    if not ds.is_dir():
        continue
    f1_to_avg = []
    try:
        for fold in range(1, 6):
            conf_mat = np.loadtxt(ds / 'XCM' / f'XP_{fold}' / 'confusion_matrix.txt')
            f1_to_avg.append(calculate_macro_f1(conf_mat))
        assert len(f1_to_avg) == 5
        f1_df.at[ds.name, 'XCM'] = np.mean(f1_to_avg)
    except Exception:
        pass

XCM_results_path = Path('../protoTS/XCM/results')
for ds in XCM_results_path.iterdir():
    if not ds.is_dir():
        continue
    f1_to_avg = []
    try:
        for fold in range(1, 6):
            conf_mat = np.loadtxt(ds / 'MTEX-CNN' / f'XP_{fold}' / 'confusion_matrix.txt')
            f1_to_avg.append(calculate_macro_f1(conf_mat))
        assert len(f1_to_avg) == 5
        f1_df.at[ds.name, 'MTEX-CNN'] = np.mean(f1_to_avg)
    except Exception:
        pass

tapnet_results = Path('../protoTS/tapnet/results')
for ds in tapnet_results.iterdir():
    f1_to_avg = []
    for run in ds.iterdir():
        conf_mat = np.loadtxt(run / 'confusion_matrix.txt')
        f1_to_avg.append(calculate_macro_f1(conf_mat))
    f1_df.at[ds.name, 'TapNet'] = np.mean(f1_to_avg)

# display(HTML(f1_df.to_html()))

unmodified_f1_df = f1_df.copy()
rounded_f1_df = f1_df.astype(float).round(2)
    
styled_f1_df = rounded_f1_df.astype(float).round(2).style.apply(display_highlight_max, axis=1)
display(HTML(styled_f1_df.format(lambda f: '{:.2f}'.format(f) if not np.isnan(f) else 'N/A').to_html()))

styled_f1_df = rounded_f1_df.apply(latex_bold_max, axis=1)

styled_f1_df = pd.DataFrame(styled_f1_df.tolist(), columns=rounded_f1_df.columns)
styled_f1_df.index = rounded_f1_df.index
styled_f1_df = styled_f1_df.sort_index(key=lambda x: x.str.lower())

latex_f1_str = styled_f1_df.to_latex(float_format='%.2f', escape=False)

print(latex_f1_str.replace('\n', '\n        '))

,ProtoTSNet,ProtoTSNet (R),MR-PETSC,XCM,MTEX-CNN,TapNet
ArticularyWordRecognition,0.97,0.96,0.99,0.71,0.94,0.98
AtrialFibrillation,0.21,0.17,0.32,0.24,0.29,0.21
BasicMotions,0.97,0.97,0.95,0.95,0.97,1.00
CharacterTrajectories,0.97,0.95,0.93,0.96,0.96,1.00
Cricket,0.93,0.95,0.99,0.68,0.89,0.98
DuckDuckGeese,0.65,0.55,0.42,0.26,0.48,0.46
EigenWorms,0.34,0.27,N/A,0.11,0.12,0.28
Epilepsy,0.98,0.84,0.98,0.89,0.93,0.97
ERing,0.90,0.82,0.88,0.09,0.84,0.82
EthanolConcentration,0.18,0.21,0.49,0.26,0.16,0.24


\begin{tabular}{lllllll}
        \toprule
         & ProtoTSNet & ProtoTSNet (R) & MR-PETSC & XCM & MTEX-CNN & TapNet \\
        \midrule
        ArticularyWordRecognition & 0.97 & 0.96 & \underline{\textbf{0.99}} & 0.71 & 0.94 & 0.98 \\
        AtrialFibrillation & 0.21 & 0.17 & \underline{\textbf{0.32}} & 0.24 & 0.29 & 0.21 \\
        BasicMotions & \textbf{0.97} & \textbf{0.97} & 0.95 & 0.95 & \textbf{0.97} & \underline{1.0} \\
        CharacterTrajectories & \textbf{0.97} & 0.95 & 0.93 & 0.96 & 0.96 & \underline{1.0} \\
        Cricket & 0.93 & 0.95 & \underline{\textbf{0.99}} & 0.68 & 0.89 & 0.98 \\
        DuckDuckGeese & \underline{\textbf{0.65}} & 0.55 & 0.42 & 0.26 & 0.48 & 0.46 \\
        EigenWorms & \underline{\textbf{0.34}} & 0.27 & N/A & 0.11 & 0.12 & 0.28 \\
        Epilepsy & \underline{\textbf{0.98}} & 0.84 & \underline{\textbf{0.98}} & 0.89 & 0.93 & 0.97 \\
        ERing & \underline{\textbf{0.9}} & 0.82 & 0.88 & 0.09 & 0.84 & 0.82 \\
        EthanolConcentration & 0.

In [6]:
ranks_f1_df = unmodified_f1_df.rank(axis=1, method='min', ascending=False)
overall_f1_ranks_mean = ranks_f1_df.mean().round(2)
wins = (ranks_f1_df == 1).sum()

explainable_f1_df = unmodified_f1_df.copy()
del explainable_f1_df['TapNet']
ranks_f1_df_ex = explainable_f1_df.rank(axis=1, method='min', ascending=False)
explainable_f1_ranks_mean = ranks_f1_df_ex.mean().round(2)
wins_ex = (ranks_f1_df_ex == 1).sum()

summary_ranks_f1_df = pd.concat([pd.DataFrame(overall_f1_ranks_mean).T, pd.DataFrame(wins).T, pd.DataFrame(explainable_f1_ranks_mean).T, pd.DataFrame(wins_ex).T])
summary_ranks_f1_df.index = ['Avg. Rank', 'Wins/Ties', 'Avg. Rank (Explainable)', 'Wins/Ties (Explainable)']
display(HTML(summary_ranks_f1_df.to_html(float_format='%.2f')))

latex_rank_df = pd.DataFrame(columns=unmodified_f1_df.columns)
latex_rank_df.loc['Avg. Rank'] = ranks_f1_df.mean()
latex_rank_df.loc['Wins/Ties'] = wins

latex_rank_df_ex = pd.DataFrame(columns=unmodified_f1_df.columns)
latex_rank_df_ex.loc['Avg. Rank'] = ranks_f1_df_ex.mean()
latex_rank_df_ex.loc['Wins/Ties'] = wins_ex

def get_bolder(do_underline):
    def bold_best(s):
        explainable = s.copy()
        del explainable['TapNet']
        non_ex_ind_st = -1
        if 'ROCKET' in explainable:
            del explainable['ROCKET']
            non_ex_ind_st = -2
        best_ex = explainable.min() if s.name == 'Avg. Rank' else explainable.max()
        is_best_ex = s == best_ex
        is_best_ex[non_ex_ind_st:] = False
        best_overall = s.min() if s.name == 'Avg. Rank' else s.max()
        is_best_overall = s == best_overall
        
        def format_float(f):
            return '{:.2f}'.format(f) if s.name == 'Avg. Rank' else '{:.0f}'.format(f)
        
        def wrap(v, bold, underline):
            v = format_float(v)
            if bold and not do_underline:
                v = '\\textbf{' + v + '}'
            if underline and do_underline:
                v = '\\underline{' + v + '}'
            return v
        return ['' if np.isnan(v) else wrap(v, v_is_ex_max, v_is_max) for v, v_is_max, v_is_ex_max in zip(s, is_best_overall, is_best_ex)]
    return bold_best

styled_df = latex_rank_df.apply(get_bolder(do_underline=True), axis=1)

styled_df = pd.DataFrame(styled_df.tolist(), columns=unmodified_f1_df.columns)
styled_df.index = latex_rank_df.index
styled_df = styled_df.sort_index(key=lambda x: x.str.lower())

latex_str = styled_df.to_latex(float_format='%.2f', escape=False)
print(latex_str.replace('\n', '\n        '))

styled_df = latex_rank_df_ex.apply(get_bolder(do_underline=False), axis=1)

styled_df = pd.DataFrame(styled_df.tolist(), columns=unmodified_f1_df.columns)
styled_df.index = latex_rank_df_ex.index
styled_df = styled_df.sort_index(key=lambda x: x.str.lower())

latex_str = styled_df.to_latex(float_format='%.2f', escape=False)
print(latex_str.replace('\n', '\n        '))


,ProtoTSNet,ProtoTSNet (R),MR-PETSC,XCM,MTEX-CNN,TapNet
Avg. Rank,2.67,3.30,3.04,4.33,4.33,2.83
Wins/Ties,7.00,6.00,7.00,1.00,1.00,8.00
Avg. Rank (Explainable),2.17,2.73,2.56,3.63,3.50,NaN
Wins/Ties (Explainable),11.00,7.00,9.00,2.00,1.00,NaN


\begin{tabular}{lllllll}
        \toprule
         & ProtoTSNet & ProtoTSNet (R) & MR-PETSC & XCM & MTEX-CNN & TapNet \\
        \midrule
        Avg. Rank & \underline{2.67} & 3.30 & 3.04 & 4.33 & 4.33 & 2.83 \\
        Wins/Ties & 7 & 6 & 7 & 1 & 1 & \underline{8} \\
        \bottomrule
        \end{tabular}
        
\begin{tabular}{lllllll}
        \toprule
         & ProtoTSNet & ProtoTSNet (R) & MR-PETSC & XCM & MTEX-CNN & TapNet \\
        \midrule
        Avg. Rank & \textbf{2.17} & 2.73 & 2.56 & 3.63 & 3.50 &  \\
        Wins/Ties & \textbf{11} & 7 & 9 & 2 & 1 &  \\
        \bottomrule
        \end{tabular}
        


In [7]:
ranks_f1_df.mean().round(2).to_csv('ranks_f1.csv', header=False)
ranks_f1_df_ex.mean().round(2).to_csv('ranks_f1_ex.csv', header=False)